In [200]:
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='amcdonne', api_key='B3jR7xq5Lt9yK0WGDkPt')
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, Event, State
import visdcc

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df = pd.read_csv('Agents_Loan_Amount.csv')

In [201]:
print('No Loan Amount Count: ', len(df[df.loan_amount.isin(['nan'])].index))

print('Row Count Original: ', len(df.index))
df = df.dropna().reset_index()
print('Row Count After NaN Drop: ', len(df.index))

cases = list(df['Unnamed: 0'])
#loan_amount = list(df['loan_amount'])
admin_agent = list(df['admin_agent'])
print('Total Case and Admin Observations: ', len(cases) + len(admin_agent))

edge_list = []
for i in range(0, len(df.index)):
    edge_list.append((str(df.loc[i]['Unnamed: 0']), str(df.loc[i]['admin_agent']), int(df.loc[i]['loan_amount'])))
print('Total Case Admin Pairs: ', len(edge_list))

No Loan Amount Count:  1
Row Count Original:  343
Row Count After NaN Drop:  342
Total Case and Admin Observations:  684
Total Case Admin Pairs:  342


In [202]:
G = nx.Graph()

G.add_nodes_from(cases)
G.add_nodes_from(admin_agent)

G.add_weighted_edges_from(edge_list, weight='weight')
#Sets the edge attributes: nx.get_edge_attributes(G, 'weight')

print('Number of Nodes: ', G.number_of_nodes())
print('Number of Edges: ', G.number_of_edges())

pos = nx.kamada_kawai_layout(G)
#position of each node in the layout of the graph
isinstance(pos, dict)

nx.set_node_attributes(G, pos, 'pos')

print('First Node and Data: ', list(G.nodes.data())[0])
print('First Node and Data: ', list(G.edges.data())[0])
#if the number of Case:Admin pairs differes from the # of edges, there are duplicates present
# Number of nodes differs from Case + Admin count -> there are duplicate entries

Number of Nodes:  454
Number of Edges:  312
First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.73309934, 0.12329558])})
First Node and Data:  ('In re Fairway Group Holdings Corp.', 'Credit Suisse', {'weight': 30611941})


In [219]:
for n, nbrsdict in G.adjacency():
    #print(n, nbrsdict)
    #print(n, len(nbrsdict.keys()))
    nx.set_node_attributes(G, len(nbrsdict.keys()), 'connections')
print('First Node and Data: ', list(G.nodes.data())[0])

First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.73309934, 0.12329558]), 'connections': 1})


In [203]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
#print(edge_trace['x'])
#print(type(edge_trace['x']))

# loops through edges, pulls out the case name and admin name; finds each names node position 
for edge in G.edges():
    #print(edge)
    x0, y0 = G.node[edge[0]]['pos']
    #print(G.node[edge[0]]['pos'])
    x1, y1 = G.node[edge[1]]['pos']
    #print(G.node[edge[1]]['pos'])
    
    edge_trace['x'] += (x0, x1)
    edge_trace['y'] += (y0, y1)

In [212]:
node_x = []
node_y = []
color_list = []
text_list = []

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_x.append(x)
    node_y.append(y)
for n, nbrsdict in G.adjacency():
    color_list.append(len(nbrsdict.keys()))
    text_list.append('# of connections: '+str(len(nbrsdict.keys())))

node_trace = go.Scatter(
    x=node_x,
    y=node_x,
    text=text_list,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=color_list,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

In [198]:
node_trace['x']

()

In [193]:
print(type(node_trace['marker']['color']))
print(type(node_trace['text']))

<class 'tuple'>
<class 'tuple'>


TypeError: can only concatenate tuple (not "int") to tuple

In [214]:
fig = go.Figure(data=[node_trace])
py.iplot(fig, filename='nodes')

In [85]:
for s, nbrs in G.adjacency():
    print(nbrs)

{'Credit Suisse': {'weight': 55000000}}
{'Credit Suisse': {'weight': 30611941}}
{'Wilmington Trust': {'weight': 275000000}}
{'Wilmington Trust': {'weight': 30000000}}
{'CitiBank': {'weight': 500000000}}
{'CitiBank': {'weight': 200000000}}
{'CitiBank': {'weight': 100000000}}
{'Strome Mezzanine Fund': {'weight': 85971731}}
{'Cortland Capital Market Services': {'weight': 100000000}}
{'Elvis Blue Moon Holdings, LLC': {'weight': 30000000}}
{'AllianceBernstein': {'weight': 4100000}}
{'Wilmington Trust': {'weight': 30000000}}
{'Textile Management Associates, Inc.': {'weight': 10000000}}
{'Madison Capital Funding': {'weight': 25704313}}
{'Wells Fargo Bank': {'weight': 25000000}}
{'Subsea 7, S.A.': {'weight': 25000000}}
{'Chiyoda Corporation': {'weight': 65000000}}
{'Wilmington Savings Fund Society': {'weight': 66670000}}
{'Wells Fargo Bank': {'weight': 116000000}}
{'AT&T Services, Inc.': {'weight': 1000000}}
{'Frost Bank': {'weight': 2000000}}
{'United Bank': {'weight': 400000}}
{'Cerberus Cap

In [ ]:
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['size'].append(len(adjacencies))
    node_info = '# of connections: '+str(len(adjacencies))
    node_trace['text'].append(node_info)

In [213]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='network')